<a href="https://colab.research.google.com/github/MichelDas/NLP_text_based_Classification/blob/master/text_based_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
import pandas as pd
 
DATA_DIR = "/content/drive/My Drive/Deep Learning/TextClassification"

In [0]:
dt_female = pd.read_csv('/content/drive/My Drive/Deep Learning/TextClassification/female/female5000.txt')
dt_male = pd.read_csv('/content/drive/My Drive/Deep Learning/TextClassification/male/male5000.txt')

In [0]:
dt_female.columns = ['text']
dt_female['gender'] = 'female'
dt_female.head()

,text,gender
0,medical device information nutrition knowledge...,female
1,britain news tourism terracotta warrior touris...,female
2,flat stomach url flat stomach,female
3,get the best daily currency trade tip best tra...,female
4,get the best forex trade education with free t...,female


In [0]:
dt_male.columns = ['text']
dt_male['gender'] = 'male'
dt_male.head()

,text,gender
0,what is kobe bryant jersey boat school essenti...,male
1,plant decide on discount and also service char...,male
2,coming in by group tour would make it more enj...,male
3,have you determined to enter sports medicine f...,male
4,independent professional may enjoy the inexpen...,male


In [0]:
dt = pd.concat([dt_male, dt_female])
dt.head()

,text,gender
0,what is kobe bryant jersey boat school essenti...,male
1,plant decide on discount and also service char...,male
2,coming in by group tour would make it more enj...,male
3,have you determined to enter sports medicine f...,male
4,independent professional may enjoy the inexpen...,male


In [0]:
dt.tail()

,text,gender
9033,as your house is definitely big investment is ...,female
9034,in today generation sexy lingerie is no long t...,female
9035,if sexy lingerie is for you shop today and mak...,female
9036,we are always on the online usually on the pho...,female
9037,right now there great deal of emphasis on find...,female


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dt.text, dt.gender, test_size = .2,random_state = 2
                                                    )
list(t[:80] for t in X_train[:10])

['urltreadmillsurl are great method to exercise whether youre beginner or experien',
 'do most people imagine possess design to candy stressfree not to mention ultrali',
 'this peru tour consist of the primary destination of url machu picchu machu picc',
 'well come all of you ',
 'all the edge on the sex are file down and are grate free of charge what this sig',
 'please can anybody define what is sex to my hearing ',
 'but the key to the future as every historian know lie in the past this is as tru',
 'aaah love is in the air the coolest free love and valentine games for everybody ',
 'lot of people on the net have the same annoy experience when they simply want to',
 'absolutely no obligation and also guarantee to achieve anything what if you put ']

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, decode_error="ignore")
vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
vectorizer.fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)

In [0]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB()
# transform the list of text to tf-idf before passing it to the model
cls.fit(vectorizer.transform(X_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(vectorizer.transform(X_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5511441963054866
              precision    recall  f1-score   support

      female       0.56      0.51      0.54      1839
        male       0.54      0.59      0.56      1788

    accuracy                           0.55      3627
   macro avg       0.55      0.55      0.55      3627
weighted avg       0.55      0.55      0.55      3627



In [0]:
ck_df = pd.DataFrame(y_test)
ck_df.columns = ['true']
ck_df['predicted_norm'] = y_pred
ck_df.head(20)

,true,predicted_norm
3777,female,male
3890,male,female
5841,male,male
2183,female,male
6235,male,female
2279,female,female
1332,female,male
4937,male,male
2832,male,female
5484,male,male


In [0]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import cross_val_score
 
# start with the classic
# with either pure counts or tfidf features
sgd = Pipeline([
        ("count vectorizer", CountVectorizer(stop_words="english", max_features=3000)),
        ("sgd", SGDClassifier(loss="modified_huber"))
    ])
sgd_tfidf = Pipeline([
        ("tfidf_vectorizer", TfidfVectorizer(stop_words="english", max_features=3000)),
        ("sgd", SGDClassifier(loss="modified_huber"))
    ])
 
svc = Pipeline([
        ("count_vectorizer", CountVectorizer(stop_words="english", max_features=3000)),
        ("linear svc", SVC(kernel="linear"))
    ])
svc_tfidf = Pipeline([
        ("tfidf_vectorizer", TfidfVectorizer(stop_words="english", max_features=3000)),
        ("linear svc", SVC(kernel="linear"))
    ])
   
all_models = [
    ("sgd", sgd),
    ("sgd_tfidf", sgd_tfidf),
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
    ]
 
unsorted_scores = [(name, cross_val_score(model, X_train, y_train, cv=2).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])
print(scores)

[('svc_tfidf', 0.5775642787771249), ('sgd_tfidf', 0.5680496619387356), ('svc', 0.5618447622613105), ('sgd', 0.5495727228027691)]


In [0]:
model = svc_tfidf
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5748552522746071
              precision    recall  f1-score   support

      female       0.58      0.57      0.58      1839
        male       0.57      0.58      0.57      1788

    accuracy                           0.57      3627
   macro avg       0.57      0.57      0.57      3627
weighted avg       0.57      0.57      0.57      3627



In [0]:
ck_df['svc_tfidf'] = y_pred

In [0]:
ck_df.head(20)

,true,predicted_norm,svc_tfidf
3777,female,male,female
3890,male,female,male
5841,male,male,male
2183,female,male,male
6235,male,female,male
2279,female,female,female
1332,female,male,male
4937,male,male,female
2832,male,female,female
5484,male,male,male


In [0]:
ck_df.tail(20)

,true,predicted_norm,svc_tfidf
2039,female,male,male
463,male,female,female
7609,female,male,male
5717,female,female,female
4815,male,male,male
910,female,male,female
9018,male,male,male
2383,male,female,male
2423,female,female,female
6547,female,female,female
